# Automation using CLI 

## Prerequisites

    | The Account should have accessed to:
    * Compute Engine
    * Storage Bucket
    * Artifact Registry
    * Cloud Build
    * Cloud Run
    
    | IAM Roles
    * Editor
    * Cloud Run Admin
    * (IAM) Security Admin    

### Enable APIS

In [ ]:
# Enable Artifact Registry, Cloud Build, and Cloud Run, Vertex AI
# !gcloud services list --available
!gcloud services enable cloudbuild.googleapis.com artifactregistry.googleapis.com run.googleapis.com aiplatform.googleapis.com cloudresourcemanager.googleapis.com

## Static External IP Address, Compute Engine, Service Account, Storage Bucket

### Environment Variables

In [ ]:
# Environment Variables
DB_NAME="db"
MACHINE_TYPE="e2-micro"
REGION="us-west1"
ZONE="us-west1-a"
BOOT_DISK_SIZE="30"
TAGS="db"
FIREWALL_RULES_NAME="ports"
STATIC_IP_ADDRESS_NAME="db-static-ip-address"

### Static IP Address

In [ ]:
# Create a static external ip address
!gcloud compute addresses create $STATIC_IP_ADDRESS_NAME --region $REGION

In [ ]:
# Print the Static IP Address
# !gcloud compute addresses describe $STATIC_IP_ADDRESS_NAME --region $REGION | grep "address: " | cut -d " " -f2

In [ ]:
# STATIC_IP_ADDRESS -> $(gcloud compute addresses describe $STATIC_IP_ADDRESS_NAME --region $REGION | grep "address: " | cut -d " " -f2)

### Startup Script

In [ ]:
%%writefile startup-script.sh
# Startup-script
# In Production, it's best practice to use Cloud Storage to store startup-script
# Environment
export USER="matt" # DB_NAME
export DBPASSWORD="password"

# docs.docker.com
curl -fsSL https://get.docker.com -o get-docker.sh
sudo sh ./get-docker.sh
sudo groupadd docker
sudo usermod -aG docker $USER
newgrp docker

# Remove all running docker 
docker rm -f $(docker ps -aq)

# Create a database with volume for data storage and backup
docker run -d \
    --name postgres-sql \
    -e POSTGRES_USER=$USER \
    -e POSTGRES_PASSWORD=$DBPASSWORD \
    -v $(pwd)/data/:/var/lib/postgresql/data/ \
    -p 5000:5432 \
    postgres
docker run -p 8000:80 \
    -e 'PGADMIN_DEFAULT_EMAIL=matt@example.com' \
    -e 'PGADMIN_DEFAULT_PASSWORD=password' \
    -d dpage/pgadmin4

### With Cloud Storage

In [ ]:
BUCKET_NAME='matt-startup-script'
STARTUP_SCRIPT_BUCKET_SA='startup-script-bucket-sa'
STARTUP_SCRIPT_NAME='startup-script.sh'

In [ ]:
# Make a bucket
!gcloud storage buckets create gs://$BUCKET_NAME

In [ ]:
# Copy the file to Cloud Storage
!gcloud storage cp startup-script.sh gs://$BUCKET_NAME

In [ ]:
# Create a service account
!gcloud iam service-accounts create $STARTUP_SCRIPT_BUCKET_SA

In [ ]:
# Enable Cloud Resource Manager
# !gcloud services enable cloudresourcemanager.googleapis.com

In [ ]:
# !gcloud projects add-iam-policy-binding \
#    $(gcloud config get project) \
#    --member=serviceAccount:$STARTUP_SCRIPT_BUCKET_SA@$(gcloud config get project).iam.gserviceaccount.com \
#    --role=roles/editor

In [ ]:
# To get the IAM Policy
# !gcloud projects get-iam-policy $(gcloud config get project)

In [ ]:
# Add IAM Policy Binding to the Bucket Service Account
!gcloud projects add-iam-policy-binding \
    $(gcloud config get project) \
    --member=serviceAccount:$STARTUP_SCRIPT_BUCKET_SA@$(gcloud config get project).iam.gserviceaccount.com \
    --role=roles/storage.objectViewer

In [ ]:
# Create an instance with these specifications
!gcloud compute instances create $DB_NAME \
    --machine-type=$MACHINE_TYPE --zone=$ZONE --tags=$TAGS \
    --boot-disk-size=$BOOT_DISK_SIZE \
    --service-account=$STARTUP_SCRIPT_BUCKET_SA@$(gcloud config get project).iam.gserviceaccount.com  \
    --metadata=startup-script-url=gs://$BUCKET_NAME/$STARTUP_SCRIPT_NAME \
    --network-interface=address=$(gcloud compute addresses describe $STATIC_IP_ADDRESS_NAME --region $REGION | grep "address: " | cut -d " " -f2)

In [ ]:
# Delete the bucket
# !gcloud storage rm -r gs://$BUCKET_NAME

In [ ]:
# Delete the instance
# !gcloud compute instances delete $DB_NAME --zone=$ZONE --quiet

##### Create a firewall (GCP)

In [ ]:
# Create a firewall (GCP)
# !gcloud compute --project=$(gcloud config get project) firewall-rules create $FIREWALL_RULES_NAME \
#    --direction=INGRESS --priority=1000 --network=default --action=ALLOW --rules=tcp:5000,tcp:8000,tcp:8501,tcp:9000 --source-ranges=0.0.0.0/0 \
#   --target-tags=$TAGS

In [ ]:
# Create a firewall (GCP)
!gcloud compute --project=$(gcloud config get project) firewall-rules create $FIREWALL_RULES_NAME \
    --direction=INGRESS --priority=1000 --network=default --action=ALLOW --rules=tcp:5000 --source-ranges=0.0.0.0/0 \
    --target-tags=$TAGS

### Without Cloud Storage

In [ ]:
# Create an instance with these specifications
!gcloud compute instances create $DB_NAME \
    --machine-type=$MACHINE_TYPE --zone=$ZONE --tags=$TAGS \
    --boot-disk-size=$BOOT_DISK_SIZE \
    --no-scopes --no-service-account \
    --metadata-from-file=startup-script=startup-script.sh \
    --network-interface=address=$(gcloud compute addresses describe $STATIC_IP_ADDRESS_NAME --region $REGION | grep "address: " | cut -d " " -f2)

In [ ]:
# Run in another terminal to run SSH to the instance
!echo "gcloud compute ssh --zone $ZONE $DB_NAME"

In [ ]:
# Delete the instance
# !gcloud compute instances delete $DB_NAME --zone=$ZONE --quiet

In [ ]:
# !gcloud compute instances add-access-config $DB_NAME --zone=$ZONE --address=$STATIC_IP_ADDRESS

## Build, Release, and Deploy

### Create an Artifact Registry

In [ ]:
# Environment Variables For Artifact Registry and Cloud Build
REGION="us-west1"
CLOUD_BUILD_REGION="us-west2"
APP_ARTIFACT_NAME="app"
APP_NAME="app"
APP_VERSION="latest"
# PROJECT_NAME -> gcloud config get project
# import os
# os.environ['REGION']=REGION
# os.environ["APP_ARTIFACT_NAME"]=APP_ARTIFACT_NAME
# os.environ["APP_NAME"]=APP_NAME
# os.environ["APP_VERSION"]=APP_VERSION

In [ ]:
# Create a Docker repository in Artifact Registry
!gcloud artifacts repositories create $APP_ARTIFACT_NAME \
    --repository-format=docker \
    --location=$REGION \
    --description="Docker repository"

In [ ]:
# Verify
!gcloud artifacts repositories list

### Build an image using Dockerfile and submit to Artifact Registry

In [ ]:
# Check if the path is correct
!pwd

In [ ]:
cd ..

In [ ]:
cd app

In [ ]:
!gcloud builds submit \
    --region=$CLOUD_BUILD_REGION \
    --tag $REGION-docker.pkg.dev/$(gcloud config get-value project)/$APP_NAME/$APP_NAME:$APP_VERSION

### Deploy using Cloud Run

In [ ]:
# For Cloud Run Deploy, use a Service Account with Cloud Run Admin
# For Clou Run Deployed Add (Service), use a Service Account with Vertex AI User or with custom IAM Role 

In [ ]:
APP_SERVICE_ACCOUNT_NAME='app-service-account'

In [ ]:
!gcloud iam service-accounts create $APP_SERVICE_ACCOUNT_NAME

In [ ]:
# Add IAM Policy Binding to the App Service Account
!gcloud projects add-iam-policy-binding \
    $(gcloud config get project) \
    --member=serviceAccount:$APP_SERVICE_ACCOUNT_NAME@$(gcloud config get project).iam.gserviceaccount.com \
    --role=roles/aiplatform.user

In [ ]:
# !gcloud iam service-accounts add-iam-policy-binding \
#    $APP_SERVICE_ACCOUNT_NAME@$(gcloud config get project).iam.gserviceaccount.com \
#    --member='serviceAccount:$APP_SERVICE_ACCOUNT_NAME@$(gcloud config get project).iam.gserviceaccount.com' \
#    --role='roles/editor'

In [ ]:
# DB Credentials

In [ ]:
!pwd

In [ ]:
cd ..

In [ ]:
cd sh-files

In [ ]:
%%writefile env.yaml
DBNAME:
    "matt"
USER:
    "matt"
HOST:
    ""
DBPORT:
    "5000"
DBPASSWORD:
    "password"
PROJECT_NAME:
    ""

In [ ]:
!gcloud run deploy $APP_NAME \
    --max-instances=1 --min-instances=1 --port=9000 \
    --env-vars-file=env.yaml \
    --image=$REGION-docker.pkg.dev/$(gcloud config get project)/$APP_NAME/$APP_NAME:$APP_VERSION \
    --allow-unauthenticated \
    --region=$REGION \
    --service-account=$APP_SERVICE_ACCOUNT_NAME@$(gcloud config get project).iam.gserviceaccount.com 

In [ ]:
!gcloud run services list

## Delete Resources

In [ ]:
!gcloud compute instances delete $DB_NAME --zone=$ZONE --quiet

In [ ]:
!gcloud compute addresses delete $STATIC_IP_ADDRESS_NAME --region $REGION --quiet

In [ ]:
!gcloud compute firewall-rules delete $FIREWALL_RULES_NAME --quiet

In [ ]:
!gcloud artifacts repositories delete $APP_ARTIFACT_NAME --location=$REGION --quiet

In [ ]:
!gcloud run services delete $APP_NAME --region=$REGION --quiet

In [ ]:
!gcloud iam service-accounts delete $STARTUP_SCRIPT_BUCKET_SA@$(gcloud config get project).iam.gserviceaccount.com --quiet

In [ ]:
!gcloud iam service-accounts delete $APP_SERVICE_ACCOUNT_NAME@$(gcloud config get project).iam.gserviceaccount.com --quiet

In [ ]:
!gcloud storage rm -r gs://$BUCKET_NAME

In [ ]:
!gcloud storage rm -r gs://$(gcloud config get project)_cloudbuild